# Softmax exercise

*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission. For more details see the [assignments page](https://compsci682-fa19.github.io/assignments2019/assignment1/) on the course website.*

This exercise is analogous to the SVM exercise. You will:

- implement a fully-vectorized **loss function** for the Softmax classifier
- implement the fully-vectorized expression for its **analytic gradient**
- **check your implementation** with numerical gradient
- use a validation set to **tune the learning rate and regularization** strength
- **optimize** the loss function with **SGD**
- **visualize** the final learned weights


In [1]:
from __future__ import print_function
import random
import numpy as np
from cs682.data_utils import load_CIFAR10
import matplotlib.pyplot as plt


%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000, num_dev=500):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the linear classifier. These are the same steps as we used for the
    SVM, but condensed to a single function.  
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = 'cs682/datasets/cifar-10-batches-py'
    
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
    
    # subsample the data
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]
    mask = np.random.choice(num_training, num_dev, replace=False)
    X_dev = X_train[mask]
    y_dev = y_train[mask]
    
    # Preprocessing: reshape the image data into rows
    X_train = np.reshape(X_train, (X_train.shape[0], -1))
    X_val = np.reshape(X_val, (X_val.shape[0], -1))
    X_test = np.reshape(X_test, (X_test.shape[0], -1))
    X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))
    
    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis = 0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image
    X_dev -= mean_image
    
    # add bias dimension and transform into columns
    X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
    X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
    X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
    X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])
    
    return X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev


# Cleaning up variables to prevent loading data multiple times (which may cause memory issue)
try:
    del X_train, y_train
    del X_test, y_test
    print('Clear previously loaded data.')
except:
    pass

# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)
print('dev data shape: ', X_dev.shape)
print('dev labels shape: ', y_dev.shape)

Train data shape:  (49000, 3073)
Train labels shape:  (49000,)
Validation data shape:  (1000, 3073)
Validation labels shape:  (1000,)
Test data shape:  (1000, 3073)
Test labels shape:  (1000,)
dev data shape:  (500, 3073)
dev labels shape:  (500,)


## Softmax Classifier

Your code for this section will all be written inside **cs682/classifiers/softmax.py**. 


In [3]:
from cs682.classifiers.softmax import softmax_loss_vectorized

# Generate a random softmax weight matrix and use it to compute the loss.
W = np.random.randn(3073, 10) * 0.0001
loss, grad = softmax_loss_vectorized(W, X_dev, y_dev, 0.000005)
print('vectorized loss: %e' % loss)

vectorized loss: 2.317764e+00


**Inline Question 1**:
Why do we expect our loss to be close to -log(0.1)? Explain briefly.**

**Your answer:** *
For the first training as we are using all random values of weights we cannot predict the correct classes, the probability of choosing a any class is equally likely. Since we have 10 classes, probability of each class being selected is (1/10) = 0.1, Therefore the softmax loss is -log(0.1).
*


In [4]:
# Complete the implementation of softmax_loss_naive and implement a (naive)
# version of the gradient that uses nested loops.
loss, grad = softmax_loss_vectorized(W, X_dev, y_dev, 0.0)

# As we did for the SVM, use numeric gradient checking as a debugging tool.
# The numeric gradient should be close to the analytic gradient.
from cs682.gradient_check import grad_check_sparse
f = lambda w: softmax_loss_vectorized(w, X_dev, y_dev, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

# similar to SVM case, do another gradient check with regularization
loss, grad = softmax_loss_vectorized(W, X_dev, y_dev, 5e1)
f = lambda w: softmax_loss_vectorized(w, X_dev, y_dev, 5e1)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

numerical: -0.457257 analytic: -0.457256, relative error: 5.720355e-08
numerical: -1.036240 analytic: -1.036240, relative error: 2.702391e-08
numerical: -1.052817 analytic: -1.052817, relative error: 9.328322e-09
numerical: -1.536259 analytic: -1.536259, relative error: 4.813092e-08
numerical: 2.683072 analytic: 2.683072, relative error: 5.074499e-09
numerical: -2.866962 analytic: -2.866963, relative error: 3.100460e-08
numerical: 0.653339 analytic: 0.653339, relative error: 4.381499e-09
numerical: -2.142865 analytic: -2.142865, relative error: 2.876715e-08
numerical: 3.194248 analytic: 3.194248, relative error: 6.020980e-09
numerical: 3.380292 analytic: 3.380292, relative error: 1.433811e-08
numerical: 2.076047 analytic: 2.076047, relative error: 4.958809e-08
numerical: -2.935238 analytic: -2.935238, relative error: 2.546502e-09
numerical: -0.740246 analytic: -0.740246, relative error: 4.163258e-08
numerical: 1.288290 analytic: 1.288290, relative error: 2.559539e-08
numerical: -1.1525

In [6]:
from cs682.classifiers.softmax import softmax_loss_naive

# Generate a random softmax weight matrix and use it to compute the loss.
W = np.random.randn(3073, 10) * 0.0001
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As a rough sanity check, our loss should be something close to -log(0.1).
print('loss: %f' % loss)

loss: 2.287954


In [6]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths; if you are careful you should be able to
# get a classification accuracy of over 0.35 on the validation set.
from cs682.classifiers import Softmax
results = {}
best_val = -1
best_softmax = None
learning_rates = [1e-7, 2.5e-7, 4e-7, 5e-7]
regularization_strengths = [2.5e4, 3.3e4, 4.3e4, 5e4]

################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained softmax classifer in best_softmax.                          #
################################################################################
for l_rate in learning_rates:
    for r_strength in regularization_strengths:
        # train svm
        softmax = Softmax()
        loss_hist = softmax.train(X_train, y_train, learning_rate=l_rate, reg=r_strength,
                      num_iters=1000, verbose=False)
        
        # predict for the validation set
        y_train_pred = softmax.predict(X_train)
        train_accuracy = np.mean(y_train == y_train_pred)
        
        # predict for the validation set
        y_val_pred = softmax.predict(X_val)
        val_accuracy = np.mean(y_val == y_val_pred)
        
        # table the result
        results[(l_rate,r_strength)] = (train_accuracy,val_accuracy)
        if val_accuracy> best_val:
            best_val = val_accuracy
            best_softmax = softmax
################################################################################
#                              END OF YOUR CODE                                #
################################################################################
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

lr 1.000000e-07 reg 2.500000e+04 train accuracy: 0.323204 val accuracy: 0.341000
lr 1.000000e-07 reg 3.300000e+04 train accuracy: 0.324510 val accuracy: 0.335000
lr 1.000000e-07 reg 4.300000e+04 train accuracy: 0.306980 val accuracy: 0.323000
lr 1.000000e-07 reg 5.000000e+04 train accuracy: 0.301408 val accuracy: 0.314000
lr 2.500000e-07 reg 2.500000e+04 train accuracy: 0.332918 val accuracy: 0.347000
lr 2.500000e-07 reg 3.300000e+04 train accuracy: 0.314939 val accuracy: 0.329000
lr 2.500000e-07 reg 4.300000e+04 train accuracy: 0.303878 val accuracy: 0.327000
lr 2.500000e-07 reg 5.000000e+04 train accuracy: 0.307020 val accuracy: 0.325000
lr 4.000000e-07 reg 2.500000e+04 train accuracy: 0.324347 val accuracy: 0.338000
lr 4.000000e-07 reg 3.300000e+04 train accuracy: 0.322286 val accuracy: 0.341000
lr 4.000000e-07 reg 4.300000e+04 train accuracy: 0.302694 val accuracy: 0.309000
lr 4.000000e-07 reg 5.000000e+04 train accuracy: 0.310776 val accuracy: 0.324000
lr 5.000000e-07 reg 2.500000

In [7]:
# evaluate on test set
# Evaluate the best softmax on test set
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('softmax on raw pixels final test set accuracy: %f' % (test_accuracy, ))

softmax on raw pixels final test set accuracy: 0.339000


Modified Loss

M2 Loss

In [3]:
from cs682.classifiers.softmax import softmax_loss_m2

# Generate a random softmax weight matrix and use it to compute the loss.
W = np.random.randn(3073, 10) * 0.0001
loss, grad = softmax_loss_m2(W, X_dev, y_dev, 0.0)

# As a rough sanity check, our loss should be something close to -log(0.1).
print('loss: %f' % loss)

loss: 3.077368


In [5]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths; if you are careful you should be able to
# get a classification accuracy of over 0.35 on the validation set.
from cs682.classifiers import Softmax_m2
results = {}
best_val = -1
best_softmax = None
learning_rates = [1e-7, 2.5e-7, 4e-7, 5e-7]
regularization_strengths = [2.5e4, 3.3e4, 4.3e4, 5e4]

################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained softmax classifer in best_softmax.                          #
################################################################################
for l_rate in learning_rates:
    for r_strength in regularization_strengths:
        # train svm
        softmax = Softmax_m2()
        loss_hist = softmax.train(X_train, y_train, learning_rate=l_rate, reg=r_strength,
                      num_iters=1000, verbose=False)
        
        # predict for the validation set
        y_train_pred = softmax.predict(X_train)
        train_accuracy = np.mean(y_train == y_train_pred)
        
        # predict for the validation set
        y_val_pred = softmax.predict(X_val)
        val_accuracy = np.mean(y_val == y_val_pred)
        
        # table the result
        results[(l_rate,r_strength)] = (train_accuracy,val_accuracy)
        if val_accuracy> best_val:
            best_val = val_accuracy
            best_softmax = softmax
################################################################################
#                              END OF YOUR CODE                                #
################################################################################
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

lr 1.000000e-07 reg 2.500000e+04 train accuracy: 0.335388 val accuracy: 0.351000
lr 1.000000e-07 reg 3.300000e+04 train accuracy: 0.318510 val accuracy: 0.341000
lr 1.000000e-07 reg 4.300000e+04 train accuracy: 0.317959 val accuracy: 0.330000
lr 1.000000e-07 reg 5.000000e+04 train accuracy: 0.300490 val accuracy: 0.321000
lr 2.500000e-07 reg 2.500000e+04 train accuracy: 0.305388 val accuracy: 0.315000
lr 2.500000e-07 reg 3.300000e+04 train accuracy: 0.305612 val accuracy: 0.310000
lr 2.500000e-07 reg 4.300000e+04 train accuracy: 0.308796 val accuracy: 0.313000
lr 2.500000e-07 reg 5.000000e+04 train accuracy: 0.309776 val accuracy: 0.333000
lr 4.000000e-07 reg 2.500000e+04 train accuracy: 0.295633 val accuracy: 0.305000
lr 4.000000e-07 reg 3.300000e+04 train accuracy: 0.312388 val accuracy: 0.331000
lr 4.000000e-07 reg 4.300000e+04 train accuracy: 0.315551 val accuracy: 0.327000
lr 4.000000e-07 reg 5.000000e+04 train accuracy: 0.306041 val accuracy: 0.329000
lr 5.000000e-07 reg 2.500000

In [6]:
# evaluate on test set
# Evaluate the best softmax on test set
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('softmax on raw pixels final test set accuracy: %f' % (test_accuracy, ))

softmax on raw pixels final test set accuracy: 0.331000


M3 Loss

In [7]:
from cs682.classifiers.softmax import softmax_loss_m3

# Generate a random softmax weight matrix and use it to compute the loss.
W = np.random.randn(3073, 10) * 0.0001
loss, grad = softmax_loss_m3(W, X_dev, y_dev, 0.0)

# As a rough sanity check, our loss should be something close to -log(0.1).
print('loss: %f' % loss)

loss: 2.382990


In [8]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths; if you are careful you should be able to
# get a classification accuracy of over 0.35 on the validation set.
from cs682.classifiers import Softmax_m3
results = {}
best_val = -1
best_softmax = None
learning_rates = [1e-7, 2.5e-7, 4e-7, 5e-7]
regularization_strengths = [2.5e4, 3.3e4, 4.3e4, 5e4]

################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained softmax classifer in best_softmax.                          #
################################################################################
for l_rate in learning_rates:
    for r_strength in regularization_strengths:
        # train svm
        softmax = Softmax_m3()
        loss_hist = softmax.train(X_train, y_train, learning_rate=l_rate, reg=r_strength,
                      num_iters=1000, verbose=False)
        
        # predict for the validation set
        y_train_pred = softmax.predict(X_train)
        train_accuracy = np.mean(y_train == y_train_pred)
        
        # predict for the validation set
        y_val_pred = softmax.predict(X_val)
        val_accuracy = np.mean(y_val == y_val_pred)
        
        # table the result
        results[(l_rate,r_strength)] = (train_accuracy,val_accuracy)
        if val_accuracy> best_val:
            best_val = val_accuracy
            best_softmax = softmax
################################################################################
#                              END OF YOUR CODE                                #
################################################################################
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

lr 1.000000e-07 reg 2.500000e+04 train accuracy: 0.337510 val accuracy: 0.335000
lr 1.000000e-07 reg 3.300000e+04 train accuracy: 0.291449 val accuracy: 0.297000
lr 1.000000e-07 reg 4.300000e+04 train accuracy: 0.279367 val accuracy: 0.291000
lr 1.000000e-07 reg 5.000000e+04 train accuracy: 0.286980 val accuracy: 0.289000
lr 2.500000e-07 reg 2.500000e+04 train accuracy: 0.281816 val accuracy: 0.290000
lr 2.500000e-07 reg 3.300000e+04 train accuracy: 0.273286 val accuracy: 0.284000
lr 2.500000e-07 reg 4.300000e+04 train accuracy: 0.298306 val accuracy: 0.302000
lr 2.500000e-07 reg 5.000000e+04 train accuracy: 0.284347 val accuracy: 0.288000
lr 4.000000e-07 reg 2.500000e+04 train accuracy: 0.277061 val accuracy: 0.290000
lr 4.000000e-07 reg 3.300000e+04 train accuracy: 0.274408 val accuracy: 0.283000
lr 4.000000e-07 reg 4.300000e+04 train accuracy: 0.301469 val accuracy: 0.324000
lr 4.000000e-07 reg 5.000000e+04 train accuracy: 0.279000 val accuracy: 0.299000
lr 5.000000e-07 reg 2.500000

In [9]:
# evaluate on test set
# Evaluate the best softmax on test set
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('softmax on raw pixels final test set accuracy: %f' % (test_accuracy, ))

softmax on raw pixels final test set accuracy: 0.339000


M4 loss

In [3]:
from cs682.classifiers.softmax import softmax_loss_m4

# Generate a random softmax weight matrix and use it to compute the loss.
W = np.random.randn(3073, 10) * 0.0001
loss, grad = softmax_loss_m4(W, X_dev, y_dev, 0.0)

# As a rough sanity check, our loss should be something close to -log(0.1).
print('loss: %f' % loss)

loss: 2.634102


In [4]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths; if you are careful you should be able to
# get a classification accuracy of over 0.35 on the validation set.
from cs682.classifiers import Softmax_m4
results = {}
best_val = -1
best_softmax = None
learning_rates = [1e-7, 2.5e-7, 4e-7, 5e-7]
regularization_strengths = [2.5e4, 3.3e4, 4.3e4, 5e4]

################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained softmax classifer in best_softmax.                          #
################################################################################
for l_rate in learning_rates:
    for r_strength in regularization_strengths:
        # train svm
        softmax = Softmax_m4()
        loss_hist = softmax.train(X_train, y_train, learning_rate=l_rate, reg=r_strength,
                      num_iters=1000, verbose=False)
        
        # predict for the validation set
        y_train_pred = softmax.predict(X_train)
        train_accuracy = np.mean(y_train == y_train_pred)
        
        # predict for the validation set
        y_val_pred = softmax.predict(X_val)
        val_accuracy = np.mean(y_val == y_val_pred)
        
        # table the result
        results[(l_rate,r_strength)] = (train_accuracy,val_accuracy)
        if val_accuracy> best_val:
            best_val = val_accuracy
            best_softmax = softmax
################################################################################
#                              END OF YOUR CODE                                #
################################################################################
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

lr 1.000000e-07 reg 2.500000e+04 train accuracy: 0.337490 val accuracy: 0.332000
lr 1.000000e-07 reg 3.300000e+04 train accuracy: 0.309388 val accuracy: 0.323000
lr 1.000000e-07 reg 4.300000e+04 train accuracy: 0.277510 val accuracy: 0.290000
lr 1.000000e-07 reg 5.000000e+04 train accuracy: 0.286388 val accuracy: 0.280000
lr 2.500000e-07 reg 2.500000e+04 train accuracy: 0.304204 val accuracy: 0.312000
lr 2.500000e-07 reg 3.300000e+04 train accuracy: 0.272878 val accuracy: 0.278000
lr 2.500000e-07 reg 4.300000e+04 train accuracy: 0.275959 val accuracy: 0.272000
lr 2.500000e-07 reg 5.000000e+04 train accuracy: 0.279837 val accuracy: 0.289000
lr 4.000000e-07 reg 2.500000e+04 train accuracy: 0.258286 val accuracy: 0.270000
lr 4.000000e-07 reg 3.300000e+04 train accuracy: 0.279531 val accuracy: 0.281000
lr 4.000000e-07 reg 4.300000e+04 train accuracy: 0.287204 val accuracy: 0.300000
lr 4.000000e-07 reg 5.000000e+04 train accuracy: 0.268449 val accuracy: 0.281000
lr 5.000000e-07 reg 2.500000

In [5]:
# evaluate on test set
# Evaluate the best softmax on test set
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('softmax on raw pixels final test set accuracy: %f' % (test_accuracy, ))

softmax on raw pixels final test set accuracy: 0.351000


COmbine vectorized and m2

In [6]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths; if you are careful you should be able to
# get a classification accuracy of over 0.35 on the validation set.
from cs682.classifiers import Softmax_combine
results = {}
best_val = -1
best_softmax = None
learning_rates = [1e-7, 2.5e-7, 4e-7, 5e-7]
regularization_strengths = [2.5e4, 3.3e4, 4.3e4, 5e4]

################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained softmax classifer in best_softmax.                          #
################################################################################
for l_rate in learning_rates:
    for r_strength in regularization_strengths:
        # train svm
        softmax = Softmax_combine()
        loss_hist = softmax.train(X_train, y_train, learning_rate=l_rate, reg=r_strength,
                      num_iters=1000, verbose=False)
        
        # predict for the validation set
        y_train_pred = softmax.predict(X_train)
        train_accuracy = np.mean(y_train == y_train_pred)
        
        # predict for the validation set
        y_val_pred = softmax.predict(X_val)
        val_accuracy = np.mean(y_val == y_val_pred)
        
        # table the result
        results[(l_rate,r_strength)] = (train_accuracy,val_accuracy)
        if val_accuracy> best_val:
            best_val = val_accuracy
            best_softmax = softmax
################################################################################
#                              END OF YOUR CODE                                #
################################################################################
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

lr 1.000000e-07 reg 2.500000e+04 train accuracy: 0.328408 val accuracy: 0.327000
lr 1.000000e-07 reg 3.300000e+04 train accuracy: 0.324306 val accuracy: 0.329000
lr 1.000000e-07 reg 4.300000e+04 train accuracy: 0.316224 val accuracy: 0.327000
lr 1.000000e-07 reg 5.000000e+04 train accuracy: 0.314612 val accuracy: 0.323000
lr 2.500000e-07 reg 2.500000e+04 train accuracy: 0.322408 val accuracy: 0.330000
lr 2.500000e-07 reg 3.300000e+04 train accuracy: 0.327673 val accuracy: 0.336000
lr 2.500000e-07 reg 4.300000e+04 train accuracy: 0.317224 val accuracy: 0.338000
lr 2.500000e-07 reg 5.000000e+04 train accuracy: 0.307694 val accuracy: 0.323000
lr 4.000000e-07 reg 2.500000e+04 train accuracy: 0.322082 val accuracy: 0.324000
lr 4.000000e-07 reg 3.300000e+04 train accuracy: 0.325469 val accuracy: 0.338000
lr 4.000000e-07 reg 4.300000e+04 train accuracy: 0.315816 val accuracy: 0.318000
lr 4.000000e-07 reg 5.000000e+04 train accuracy: 0.311980 val accuracy: 0.309000
lr 5.000000e-07 reg 2.500000

In [7]:
# evaluate on test set
# Evaluate the best softmax on test set
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('softmax on raw pixels final test set accuracy: %f' % (test_accuracy, ))

softmax on raw pixels final test set accuracy: 0.326000


Vectorized + m3

In [3]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths; if you are careful you should be able to
# get a classification accuracy of over 0.35 on the validation set.
from cs682.classifiers import *
results = {}
best_val = -1
best_softmax = None
learning_rates = [1e-7, 2.5e-7, 4e-7, 5e-7]
regularization_strengths = [2.5e4, 3.3e4, 4.3e4, 5e4]

################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained softmax classifer in best_softmax.                          #
################################################################################
for l_rate in learning_rates:
    for r_strength in regularization_strengths:
        # train svm
        softmax = Softmax_combinem3()
        loss_hist = softmax.train(X_train, y_train, learning_rate=l_rate, reg=r_strength,
                      num_iters=1000, verbose=False)
        
        # predict for the validation set
        y_train_pred = softmax.predict(X_train)
        train_accuracy = np.mean(y_train == y_train_pred)
        
        # predict for the validation set
        y_val_pred = softmax.predict(X_val)
        val_accuracy = np.mean(y_val == y_val_pred)
        
        # table the result
        results[(l_rate,r_strength)] = (train_accuracy,val_accuracy)
        if val_accuracy> best_val:
            best_val = val_accuracy
            best_softmax = softmax
################################################################################
#                              END OF YOUR CODE                                #
################################################################################
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

C:\Users\sneha\682Project\assignment1\cs682\classifiers\softmax.py:188: RuntimeWarning: divide by zero encountered in log
  loss += -np.log(softmax_prob[y[i]])


lr 1.000000e-07 reg 2.500000e+04 train accuracy: 0.313673 val accuracy: 0.319000
lr 1.000000e-07 reg 3.300000e+04 train accuracy: 0.307245 val accuracy: 0.309000
lr 1.000000e-07 reg 4.300000e+04 train accuracy: 0.302082 val accuracy: 0.310000
lr 1.000000e-07 reg 5.000000e+04 train accuracy: 0.304551 val accuracy: 0.310000
lr 2.500000e-07 reg 2.500000e+04 train accuracy: 0.310265 val accuracy: 0.316000
lr 2.500000e-07 reg 3.300000e+04 train accuracy: 0.324510 val accuracy: 0.324000
lr 2.500000e-07 reg 4.300000e+04 train accuracy: 0.293061 val accuracy: 0.300000
lr 2.500000e-07 reg 5.000000e+04 train accuracy: 0.273122 val accuracy: 0.275000
lr 4.000000e-07 reg 2.500000e+04 train accuracy: 0.305653 val accuracy: 0.310000
lr 4.000000e-07 reg 3.300000e+04 train accuracy: 0.290980 val accuracy: 0.310000
lr 4.000000e-07 reg 4.300000e+04 train accuracy: 0.301184 val accuracy: 0.307000
lr 4.000000e-07 reg 5.000000e+04 train accuracy: 0.289980 val accuracy: 0.283000
lr 5.000000e-07 reg 2.500000

In [4]:
# evaluate on test set
# Evaluate the best softmax on test set
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('softmax on raw pixels final test set accuracy: %f' % (test_accuracy, ))

softmax on raw pixels final test set accuracy: 0.346000


Vectorized + m4

In [3]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths; if you are careful you should be able to
# get a classification accuracy of over 0.35 on the validation set.
from cs682.classifiers import *
results = {}
best_val = -1
best_softmax = None
learning_rates = [1e-7, 2.5e-7, 4e-7, 5e-7]
regularization_strengths = [2.5e4, 3.3e4, 4.3e4, 5e4]

################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained softmax classifer in best_softmax.                          #
################################################################################
for l_rate in learning_rates:
    for r_strength in regularization_strengths:
        # train svm
        softmax = Softmax_combinem4()
        loss_hist = softmax.train(X_train, y_train, learning_rate=l_rate, reg=r_strength,
                      num_iters=1000, verbose=False)
        
        # predict for the validation set
        y_train_pred = softmax.predict(X_train)
        train_accuracy = np.mean(y_train == y_train_pred)
        
        # predict for the validation set
        y_val_pred = softmax.predict(X_val)
        val_accuracy = np.mean(y_val == y_val_pred)
        
        # table the result
        results[(l_rate,r_strength)] = (train_accuracy,val_accuracy)
        if val_accuracy> best_val:
            best_val = val_accuracy
            best_softmax = softmax
################################################################################
#                              END OF YOUR CODE                                #
################################################################################
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

lr 1.000000e-07 reg 2.500000e+04 train accuracy: 0.314857 val accuracy: 0.328000
lr 1.000000e-07 reg 3.300000e+04 train accuracy: 0.309980 val accuracy: 0.320000
lr 1.000000e-07 reg 4.300000e+04 train accuracy: 0.298000 val accuracy: 0.312000
lr 1.000000e-07 reg 5.000000e+04 train accuracy: 0.286857 val accuracy: 0.302000
lr 2.500000e-07 reg 2.500000e+04 train accuracy: 0.307510 val accuracy: 0.325000
lr 2.500000e-07 reg 3.300000e+04 train accuracy: 0.301857 val accuracy: 0.312000
lr 2.500000e-07 reg 4.300000e+04 train accuracy: 0.294694 val accuracy: 0.309000
lr 2.500000e-07 reg 5.000000e+04 train accuracy: 0.279633 val accuracy: 0.298000
lr 4.000000e-07 reg 2.500000e+04 train accuracy: 0.311204 val accuracy: 0.327000
lr 4.000000e-07 reg 3.300000e+04 train accuracy: 0.292102 val accuracy: 0.303000
lr 4.000000e-07 reg 4.300000e+04 train accuracy: 0.302837 val accuracy: 0.310000
lr 4.000000e-07 reg 5.000000e+04 train accuracy: 0.281653 val accuracy: 0.292000
lr 5.000000e-07 reg 2.500000

In [4]:
# evaluate on test set
# Evaluate the best softmax on test set
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('softmax on raw pixels final test set accuracy: %f' % (test_accuracy, ))

softmax on raw pixels final test set accuracy: 0.328000


m2 + m3

In [5]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths; if you are careful you should be able to
# get a classification accuracy of over 0.35 on the validation set.
from cs682.classifiers import *
results = {}
best_val = -1
best_softmax = None
learning_rates = [1e-7, 2.5e-7, 4e-7, 5e-7]
regularization_strengths = [2.5e4, 3.3e4, 4.3e4, 5e4]

################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained softmax classifer in best_softmax.                          #
################################################################################
for l_rate in learning_rates:
    for r_strength in regularization_strengths:
        # train svm
        softmax = Softmax_m2m3()
        loss_hist = softmax.train(X_train, y_train, learning_rate=l_rate, reg=r_strength,
                      num_iters=1000, verbose=False)
        
        # predict for the validation set
        y_train_pred = softmax.predict(X_train)
        train_accuracy = np.mean(y_train == y_train_pred)
        
        # predict for the validation set
        y_val_pred = softmax.predict(X_val)
        val_accuracy = np.mean(y_val == y_val_pred)
        
        # table the result
        results[(l_rate,r_strength)] = (train_accuracy,val_accuracy)
        if val_accuracy> best_val:
            best_val = val_accuracy
            best_softmax = softmax
################################################################################
#                              END OF YOUR CODE                                #
################################################################################
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

C:\Users\sneha\682Project\assignment1\cs682\classifiers\softmax.py:188: RuntimeWarning: divide by zero encountered in log
  loss += -np.log(softmax_prob[y[i]])


lr 1.000000e-07 reg 2.500000e+04 train accuracy: 0.309327 val accuracy: 0.322000
lr 1.000000e-07 reg 3.300000e+04 train accuracy: 0.305347 val accuracy: 0.311000
lr 1.000000e-07 reg 4.300000e+04 train accuracy: 0.311592 val accuracy: 0.320000
lr 1.000000e-07 reg 5.000000e+04 train accuracy: 0.317020 val accuracy: 0.333000
lr 2.500000e-07 reg 2.500000e+04 train accuracy: 0.307551 val accuracy: 0.316000
lr 2.500000e-07 reg 3.300000e+04 train accuracy: 0.283714 val accuracy: 0.298000
lr 2.500000e-07 reg 4.300000e+04 train accuracy: 0.297796 val accuracy: 0.319000
lr 2.500000e-07 reg 5.000000e+04 train accuracy: 0.283510 val accuracy: 0.288000
lr 4.000000e-07 reg 2.500000e+04 train accuracy: 0.297918 val accuracy: 0.310000
lr 4.000000e-07 reg 3.300000e+04 train accuracy: 0.271102 val accuracy: 0.291000
lr 4.000000e-07 reg 4.300000e+04 train accuracy: 0.267510 val accuracy: 0.281000
lr 4.000000e-07 reg 5.000000e+04 train accuracy: 0.295184 val accuracy: 0.314000
lr 5.000000e-07 reg 2.500000

In [6]:
# evaluate on test set
# Evaluate the best softmax on test set
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('softmax on raw pixels final test set accuracy: %f' % (test_accuracy, ))

softmax on raw pixels final test set accuracy: 0.315000


m3 + m4

In [7]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths; if you are careful you should be able to
# get a classification accuracy of over 0.35 on the validation set.
from cs682.classifiers import *
results = {}
best_val = -1
best_softmax = None
learning_rates = [1e-7, 2.5e-7, 4e-7, 5e-7]
regularization_strengths = [2.5e4, 3.3e4, 4.3e4, 5e4]

################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained softmax classifer in best_softmax.                          #
################################################################################
for l_rate in learning_rates:
    for r_strength in regularization_strengths:
        # train svm
        softmax = Softmax_m3m4()
        loss_hist = softmax.train(X_train, y_train, learning_rate=l_rate, reg=r_strength,
                      num_iters=1000, verbose=False)
        
        # predict for the validation set
        y_train_pred = softmax.predict(X_train)
        train_accuracy = np.mean(y_train == y_train_pred)
        
        # predict for the validation set
        y_val_pred = softmax.predict(X_val)
        val_accuracy = np.mean(y_val == y_val_pred)
        
        # table the result
        results[(l_rate,r_strength)] = (train_accuracy,val_accuracy)
        if val_accuracy> best_val:
            best_val = val_accuracy
            best_softmax = softmax
################################################################################
#                              END OF YOUR CODE                                #
################################################################################
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

lr 1.000000e-07 reg 2.500000e+04 train accuracy: 0.269694 val accuracy: 0.286000
lr 1.000000e-07 reg 3.300000e+04 train accuracy: 0.297061 val accuracy: 0.303000
lr 1.000000e-07 reg 4.300000e+04 train accuracy: 0.264102 val accuracy: 0.270000
lr 1.000000e-07 reg 5.000000e+04 train accuracy: 0.297204 val accuracy: 0.305000
lr 2.500000e-07 reg 2.500000e+04 train accuracy: 0.295306 val accuracy: 0.315000
lr 2.500000e-07 reg 3.300000e+04 train accuracy: 0.282265 val accuracy: 0.289000
lr 2.500000e-07 reg 4.300000e+04 train accuracy: 0.280939 val accuracy: 0.310000
lr 2.500000e-07 reg 5.000000e+04 train accuracy: 0.243041 val accuracy: 0.254000
lr 4.000000e-07 reg 2.500000e+04 train accuracy: 0.286653 val accuracy: 0.291000
lr 4.000000e-07 reg 3.300000e+04 train accuracy: 0.287245 val accuracy: 0.300000
lr 4.000000e-07 reg 4.300000e+04 train accuracy: 0.266510 val accuracy: 0.278000
lr 4.000000e-07 reg 5.000000e+04 train accuracy: 0.242061 val accuracy: 0.244000
lr 5.000000e-07 reg 2.500000

In [8]:
# evaluate on test set
# Evaluate the best softmax on test set
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('softmax on raw pixels final test set accuracy: %f' % (test_accuracy, ))

softmax on raw pixels final test set accuracy: 0.294000


m2 + m4

In [9]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths; if you are careful you should be able to
# get a classification accuracy of over 0.35 on the validation set.
from cs682.classifiers import *
results = {}
best_val = -1
best_softmax = None
learning_rates = [1e-7, 2.5e-7, 4e-7, 5e-7]
regularization_strengths = [2.5e4, 3.3e4, 4.3e4, 5e4]

################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained softmax classifer in best_softmax.                          #
################################################################################
for l_rate in learning_rates:
    for r_strength in regularization_strengths:
        # train svm
        softmax = Softmax_m2m4()
        loss_hist = softmax.train(X_train, y_train, learning_rate=l_rate, reg=r_strength,
                      num_iters=1000, verbose=False)
        
        # predict for the validation set
        y_train_pred = softmax.predict(X_train)
        train_accuracy = np.mean(y_train == y_train_pred)
        
        # predict for the validation set
        y_val_pred = softmax.predict(X_val)
        val_accuracy = np.mean(y_val == y_val_pred)
        
        # table the result
        results[(l_rate,r_strength)] = (train_accuracy,val_accuracy)
        if val_accuracy> best_val:
            best_val = val_accuracy
            best_softmax = softmax
################################################################################
#                              END OF YOUR CODE                                #
################################################################################
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

lr 1.000000e-07 reg 2.500000e+04 train accuracy: 0.303531 val accuracy: 0.316000
lr 1.000000e-07 reg 3.300000e+04 train accuracy: 0.310082 val accuracy: 0.336000
lr 1.000000e-07 reg 4.300000e+04 train accuracy: 0.304653 val accuracy: 0.324000
lr 1.000000e-07 reg 5.000000e+04 train accuracy: 0.315408 val accuracy: 0.334000
lr 2.500000e-07 reg 2.500000e+04 train accuracy: 0.314878 val accuracy: 0.344000
lr 2.500000e-07 reg 3.300000e+04 train accuracy: 0.310327 val accuracy: 0.335000
lr 2.500000e-07 reg 4.300000e+04 train accuracy: 0.301918 val accuracy: 0.319000
lr 2.500000e-07 reg 5.000000e+04 train accuracy: 0.304980 val accuracy: 0.312000
lr 4.000000e-07 reg 2.500000e+04 train accuracy: 0.295755 val accuracy: 0.312000
lr 4.000000e-07 reg 3.300000e+04 train accuracy: 0.273122 val accuracy: 0.281000
lr 4.000000e-07 reg 4.300000e+04 train accuracy: 0.306510 val accuracy: 0.317000
lr 4.000000e-07 reg 5.000000e+04 train accuracy: 0.297551 val accuracy: 0.328000
lr 5.000000e-07 reg 2.500000

In [10]:
# evaluate on test set
# Evaluate the best softmax on test set
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('softmax on raw pixels final test set accuracy: %f' % (test_accuracy, ))

softmax on raw pixels final test set accuracy: 0.312000


Softmax + svm

In [3]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths; if you are careful you should be able to
# get a classification accuracy of over 0.35 on the validation set.
from cs682.classifiers import *
results = {}
best_val = -1
best_softmax = None
learning_rates = [1e-7, 2.5e-7, 4e-7, 5e-7]
regularization_strengths = [2.5e4, 3.3e4, 4.3e4, 5e4]

################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained softmax classifer in best_softmax.                          #
################################################################################
for l_rate in learning_rates:
    for r_strength in regularization_strengths:
        # train svm
        softmax = SVM_softmax()
        loss_hist = softmax.train(X_train, y_train, learning_rate=l_rate, reg=r_strength,
                      num_iters=1000, verbose=False)
        
        # predict for the validation set
        y_train_pred = softmax.predict(X_train)
        train_accuracy = np.mean(y_train == y_train_pred)
        
        # predict for the validation set
        y_val_pred = softmax.predict(X_val)
        val_accuracy = np.mean(y_val == y_val_pred)
        
        # table the result
        results[(l_rate,r_strength)] = (train_accuracy,val_accuracy)
        if val_accuracy> best_val:
            best_val = val_accuracy
            best_softmax = softmax
################################################################################
#                              END OF YOUR CODE                                #
################################################################################
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

lr 1.000000e-07 reg 2.500000e+04 train accuracy: 0.356694 val accuracy: 0.364000
lr 1.000000e-07 reg 3.300000e+04 train accuracy: 0.356367 val accuracy: 0.371000
lr 1.000000e-07 reg 4.300000e+04 train accuracy: 0.348388 val accuracy: 0.360000
lr 1.000000e-07 reg 5.000000e+04 train accuracy: 0.354020 val accuracy: 0.360000
lr 2.500000e-07 reg 2.500000e+04 train accuracy: 0.358286 val accuracy: 0.371000
lr 2.500000e-07 reg 3.300000e+04 train accuracy: 0.346082 val accuracy: 0.352000
lr 2.500000e-07 reg 4.300000e+04 train accuracy: 0.339469 val accuracy: 0.363000
lr 2.500000e-07 reg 5.000000e+04 train accuracy: 0.336918 val accuracy: 0.322000
lr 4.000000e-07 reg 2.500000e+04 train accuracy: 0.332694 val accuracy: 0.352000
lr 4.000000e-07 reg 3.300000e+04 train accuracy: 0.328796 val accuracy: 0.324000
lr 4.000000e-07 reg 4.300000e+04 train accuracy: 0.296367 val accuracy: 0.299000
lr 4.000000e-07 reg 5.000000e+04 train accuracy: 0.306673 val accuracy: 0.323000
lr 5.000000e-07 reg 2.500000

In [4]:
# evaluate on test set
# Evaluate the best softmax on test set
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('softmax on raw pixels final test set accuracy: %f' % (test_accuracy, ))

softmax on raw pixels final test set accuracy: 0.354000


A-Softmax

In [3]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths; if you are careful you should be able to
# get a classification accuracy of over 0.35 on the validation set.
from cs682.classifiers import *
results = {}
best_val = -1
best_softmax = None
learning_rates = [1e-7, 2.5e-7, 4e-7, 5e-7]
regularization_strengths = [2.5e4, 3.3e4, 4.3e4, 5e4]

################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained softmax classifer in best_softmax.                          #
################################################################################
for l_rate in learning_rates:
    for r_strength in regularization_strengths:
        # train svm
        softmax = ASoftmax()
        loss_hist = softmax.train(X_train, y_train, learning_rate=l_rate, reg=r_strength,
                      num_iters=1000, verbose=False)
        
        # predict for the validation set
        y_train_pred = softmax.predict(X_train)
        train_accuracy = np.mean(y_train == y_train_pred)
        
        # predict for the validation set
        y_val_pred = softmax.predict(X_val)
        val_accuracy = np.mean(y_val == y_val_pred)
        
        # table the result
        results[(l_rate,r_strength)] = (train_accuracy,val_accuracy)
        if val_accuracy> best_val:
            best_val = val_accuracy
            best_softmax = softmax
################################################################################
#                              END OF YOUR CODE                                #
################################################################################
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

lr 1.000000e-07 reg 2.500000e+04 train accuracy: 0.328837 val accuracy: 0.347000
lr 1.000000e-07 reg 3.300000e+04 train accuracy: 0.316082 val accuracy: 0.341000
lr 1.000000e-07 reg 4.300000e+04 train accuracy: 0.314939 val accuracy: 0.336000
lr 1.000000e-07 reg 5.000000e+04 train accuracy: 0.305286 val accuracy: 0.321000
lr 2.500000e-07 reg 2.500000e+04 train accuracy: 0.322163 val accuracy: 0.344000
lr 2.500000e-07 reg 3.300000e+04 train accuracy: 0.316020 val accuracy: 0.330000
lr 2.500000e-07 reg 4.300000e+04 train accuracy: 0.298735 val accuracy: 0.318000
lr 2.500000e-07 reg 5.000000e+04 train accuracy: 0.299020 val accuracy: 0.313000
lr 4.000000e-07 reg 2.500000e+04 train accuracy: 0.318959 val accuracy: 0.336000
lr 4.000000e-07 reg 3.300000e+04 train accuracy: 0.313510 val accuracy: 0.341000
lr 4.000000e-07 reg 4.300000e+04 train accuracy: 0.300163 val accuracy: 0.318000
lr 4.000000e-07 reg 5.000000e+04 train accuracy: 0.314857 val accuracy: 0.326000
lr 5.000000e-07 reg 2.500000

In [4]:
# evaluate on test set
# Evaluate the best softmax on test set
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('softmax on raw pixels final test set accuracy: %f' % (test_accuracy, ))

softmax on raw pixels final test set accuracy: 0.336000


A-Softmax m=2

In [5]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths; if you are careful you should be able to
# get a classification accuracy of over 0.35 on the validation set.
from cs682.classifiers import *
results = {}
best_val = -1
best_softmax = None
learning_rates = [1e-7, 2.5e-7, 4e-7, 5e-7]
regularization_strengths = [2.5e4, 3.3e4, 4.3e4, 5e4]

################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained softmax classifer in best_softmax.                          #
################################################################################
for l_rate in learning_rates:
    for r_strength in regularization_strengths:
        # train svm
        softmax = ASoftmax_m2()
        loss_hist = softmax.train(X_train, y_train, learning_rate=l_rate, reg=r_strength,
                      num_iters=1000, verbose=False)
        
        # predict for the validation set
        y_train_pred = softmax.predict(X_train)
        train_accuracy = np.mean(y_train == y_train_pred)
        
        # predict for the validation set
        y_val_pred = softmax.predict(X_val)
        val_accuracy = np.mean(y_val == y_val_pred)
        
        # table the result
        results[(l_rate,r_strength)] = (train_accuracy,val_accuracy)
        if val_accuracy> best_val:
            best_val = val_accuracy
            best_softmax = softmax
################################################################################
#                              END OF YOUR CODE                                #
################################################################################
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

C:\Users\sneha\682Project\assignment1\cs682\classifiers\softmax.py:399: RuntimeWarning: divide by zero encountered in log
  loss += -np.log(softmax_prob[y[i]])


lr 1.000000e-07 reg 2.500000e+04 train accuracy: 0.326204 val accuracy: 0.334000
lr 1.000000e-07 reg 3.300000e+04 train accuracy: 0.314429 val accuracy: 0.335000
lr 1.000000e-07 reg 4.300000e+04 train accuracy: 0.305837 val accuracy: 0.318000
lr 1.000000e-07 reg 5.000000e+04 train accuracy: 0.303918 val accuracy: 0.322000
lr 2.500000e-07 reg 2.500000e+04 train accuracy: 0.323898 val accuracy: 0.333000
lr 2.500000e-07 reg 3.300000e+04 train accuracy: 0.320286 val accuracy: 0.331000
lr 2.500000e-07 reg 4.300000e+04 train accuracy: 0.298776 val accuracy: 0.312000
lr 2.500000e-07 reg 5.000000e+04 train accuracy: 0.304857 val accuracy: 0.327000
lr 4.000000e-07 reg 2.500000e+04 train accuracy: 0.325449 val accuracy: 0.328000
lr 4.000000e-07 reg 3.300000e+04 train accuracy: 0.308939 val accuracy: 0.322000
lr 4.000000e-07 reg 4.300000e+04 train accuracy: 0.316367 val accuracy: 0.324000
lr 4.000000e-07 reg 5.000000e+04 train accuracy: 0.302939 val accuracy: 0.318000
lr 5.000000e-07 reg 2.500000

In [6]:
# evaluate on test set
# Evaluate the best softmax on test set
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('softmax on raw pixels final test set accuracy: %f' % (test_accuracy, ))

softmax on raw pixels final test set accuracy: 0.323000


A-Softmax m=3

In [7]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths; if you are careful you should be able to
# get a classification accuracy of over 0.35 on the validation set.
from cs682.classifiers import *
results = {}
best_val = -1
best_softmax = None
learning_rates = [1e-7, 2.5e-7, 4e-7, 5e-7]
regularization_strengths = [2.5e4, 3.3e4, 4.3e4, 5e4]

################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained softmax classifer in best_softmax.                          #
################################################################################
for l_rate in learning_rates:
    for r_strength in regularization_strengths:
        # train svm
        softmax = ASoftmax_m3()
        loss_hist = softmax.train(X_train, y_train, learning_rate=l_rate, reg=r_strength,
                      num_iters=1000, verbose=False)
        
        # predict for the validation set
        y_train_pred = softmax.predict(X_train)
        train_accuracy = np.mean(y_train == y_train_pred)
        
        # predict for the validation set
        y_val_pred = softmax.predict(X_val)
        val_accuracy = np.mean(y_val == y_val_pred)
        
        # table the result
        results[(l_rate,r_strength)] = (train_accuracy,val_accuracy)
        if val_accuracy> best_val:
            best_val = val_accuracy
            best_softmax = softmax
################################################################################
#                              END OF YOUR CODE                                #
################################################################################
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

lr 1.000000e-07 reg 2.500000e+04 train accuracy: 0.318449 val accuracy: 0.338000
lr 1.000000e-07 reg 3.300000e+04 train accuracy: 0.309265 val accuracy: 0.324000
lr 1.000000e-07 reg 4.300000e+04 train accuracy: 0.304939 val accuracy: 0.317000
lr 1.000000e-07 reg 5.000000e+04 train accuracy: 0.297571 val accuracy: 0.307000
lr 2.500000e-07 reg 2.500000e+04 train accuracy: 0.324449 val accuracy: 0.340000
lr 2.500000e-07 reg 3.300000e+04 train accuracy: 0.311776 val accuracy: 0.322000
lr 2.500000e-07 reg 4.300000e+04 train accuracy: 0.296796 val accuracy: 0.322000
lr 2.500000e-07 reg 5.000000e+04 train accuracy: 0.281612 val accuracy: 0.299000
lr 4.000000e-07 reg 2.500000e+04 train accuracy: 0.311041 val accuracy: 0.332000
lr 4.000000e-07 reg 3.300000e+04 train accuracy: 0.315571 val accuracy: 0.330000
lr 4.000000e-07 reg 4.300000e+04 train accuracy: 0.305980 val accuracy: 0.325000
lr 4.000000e-07 reg 5.000000e+04 train accuracy: 0.291327 val accuracy: 0.300000
lr 5.000000e-07 reg 2.500000

In [8]:
# evaluate on test set
# Evaluate the best softmax on test set
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('softmax on raw pixels final test set accuracy: %f' % (test_accuracy, ))

softmax on raw pixels final test set accuracy: 0.336000


A-Softmax m=4

In [9]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths; if you are careful you should be able to
# get a classification accuracy of over 0.35 on the validation set.
from cs682.classifiers import *
results = {}
best_val = -1
best_softmax = None
learning_rates = [1e-7, 2.5e-7, 4e-7, 5e-7]
regularization_strengths = [2.5e4, 3.3e4, 4.3e4, 5e4]

################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained softmax classifer in best_softmax.                          #
################################################################################
for l_rate in learning_rates:
    for r_strength in regularization_strengths:
        # train svm
        softmax = ASoftmax_m4()
        loss_hist = softmax.train(X_train, y_train, learning_rate=l_rate, reg=r_strength,
                      num_iters=1000, verbose=False)
        
        # predict for the validation set
        y_train_pred = softmax.predict(X_train)
        train_accuracy = np.mean(y_train == y_train_pred)
        
        # predict for the validation set
        y_val_pred = softmax.predict(X_val)
        val_accuracy = np.mean(y_val == y_val_pred)
        
        # table the result
        results[(l_rate,r_strength)] = (train_accuracy,val_accuracy)
        if val_accuracy> best_val:
            best_val = val_accuracy
            best_softmax = softmax
################################################################################
#                              END OF YOUR CODE                                #
################################################################################
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

C:\Users\sneha\682Project\assignment1\cs682\classifiers\softmax.py:543: RuntimeWarning: divide by zero encountered in log
  loss += -np.log(softmax_prob[y[i]])


lr 1.000000e-07 reg 2.500000e+04 train accuracy: 0.321000 val accuracy: 0.329000
lr 1.000000e-07 reg 3.300000e+04 train accuracy: 0.313796 val accuracy: 0.329000
lr 1.000000e-07 reg 4.300000e+04 train accuracy: 0.308347 val accuracy: 0.329000
lr 1.000000e-07 reg 5.000000e+04 train accuracy: 0.309367 val accuracy: 0.332000
lr 2.500000e-07 reg 2.500000e+04 train accuracy: 0.318143 val accuracy: 0.342000
lr 2.500000e-07 reg 3.300000e+04 train accuracy: 0.305490 val accuracy: 0.321000
lr 2.500000e-07 reg 4.300000e+04 train accuracy: 0.306265 val accuracy: 0.318000
lr 2.500000e-07 reg 5.000000e+04 train accuracy: 0.299755 val accuracy: 0.310000
lr 4.000000e-07 reg 2.500000e+04 train accuracy: 0.323265 val accuracy: 0.330000
lr 4.000000e-07 reg 3.300000e+04 train accuracy: 0.305469 val accuracy: 0.319000
lr 4.000000e-07 reg 4.300000e+04 train accuracy: 0.296735 val accuracy: 0.328000
lr 4.000000e-07 reg 5.000000e+04 train accuracy: 0.308204 val accuracy: 0.325000
lr 5.000000e-07 reg 2.500000

In [10]:
# evaluate on test set
# Evaluate the best softmax on test set
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('softmax on raw pixels final test set accuracy: %f' % (test_accuracy, ))

softmax on raw pixels final test set accuracy: 0.332000


A-Softmax + L-Softmax

In [3]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths; if you are careful you should be able to
# get a classification accuracy of over 0.35 on the validation set.
from cs682.classifiers import *
results = {}
best_val = -1
best_softmax = None
learning_rates = [1e-7, 2.5e-7, 4e-7, 5e-7]
regularization_strengths = [2.5e4, 3.3e4, 4.3e4, 5e4]

################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained softmax classifer in best_softmax.                          #
################################################################################
for l_rate in learning_rates:
    for r_strength in regularization_strengths:
        # train svm
        softmax = ASoftmax_Lsoftmax()
        loss_hist = softmax.train(X_train, y_train, learning_rate=l_rate, reg=r_strength,
                      num_iters=1000, verbose=False)
        
        # predict for the validation set
        y_train_pred = softmax.predict(X_train)
        train_accuracy = np.mean(y_train == y_train_pred)
        
        # predict for the validation set
        y_val_pred = softmax.predict(X_val)
        val_accuracy = np.mean(y_val == y_val_pred)
        
        # table the result
        results[(l_rate,r_strength)] = (train_accuracy,val_accuracy)
        if val_accuracy> best_val:
            best_val = val_accuracy
            best_softmax = softmax
################################################################################
#                              END OF YOUR CODE                                #
################################################################################
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

C:\Users\sneha\682Project\assignment1\cs682\classifiers\softmax.py:188: RuntimeWarning: divide by zero encountered in log
  loss += -np.log(softmax_prob[y[i]])


lr 1.000000e-07 reg 2.500000e+04 train accuracy: 0.307735 val accuracy: 0.320000
lr 1.000000e-07 reg 3.300000e+04 train accuracy: 0.306633 val accuracy: 0.327000
lr 1.000000e-07 reg 4.300000e+04 train accuracy: 0.313122 val accuracy: 0.310000
lr 1.000000e-07 reg 5.000000e+04 train accuracy: 0.292245 val accuracy: 0.301000
lr 2.500000e-07 reg 2.500000e+04 train accuracy: 0.312918 val accuracy: 0.330000
lr 2.500000e-07 reg 3.300000e+04 train accuracy: 0.290408 val accuracy: 0.302000
lr 2.500000e-07 reg 4.300000e+04 train accuracy: 0.302918 val accuracy: 0.325000
lr 2.500000e-07 reg 5.000000e+04 train accuracy: 0.306898 val accuracy: 0.310000
lr 4.000000e-07 reg 2.500000e+04 train accuracy: 0.301163 val accuracy: 0.315000
lr 4.000000e-07 reg 3.300000e+04 train accuracy: 0.283265 val accuracy: 0.307000
lr 4.000000e-07 reg 4.300000e+04 train accuracy: 0.290306 val accuracy: 0.308000
lr 4.000000e-07 reg 5.000000e+04 train accuracy: 0.287796 val accuracy: 0.296000
lr 5.000000e-07 reg 2.500000

In [4]:
# evaluate on test set
# Evaluate the best softmax on test set
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('softmax on raw pixels final test set accuracy: %f' % (test_accuracy, ))

softmax on raw pixels final test set accuracy: 0.326000


Grid Search for Loss Combination

In [ ]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths; if you are careful you should be able to
# get a classification accuracy of over 0.35 on the validation set.
from cs682.classifiers import *
results = {}
best_val = -1
best_softmax = None
learning_rates = [1e-7, 2.5e-7, 4e-7, 5e-7]
regularization_strengths = [2.5e4, 3.3e4, 4.3e4, 5e4]


################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained softmax classifer in best_softmax.                          #
################################################################################
for l_rate in learning_rates:
    for r_strength in regularization_strengths:
        # train svm
        softmax = ASoftmax_Lsoftmax()
        loss_hist = softmax.train(X_train, y_train, learning_rate=l_rate, reg=r_strength,
                      num_iters=1000, verbose=False)
        
        # predict for the validation set
        y_train_pred = softmax.predict(X_train)
        train_accuracy = np.mean(y_train == y_train_pred)
        
        # predict for the validation set
        y_val_pred = softmax.predict(X_val)
        val_accuracy = np.mean(y_val == y_val_pred)
        
        # table the result
        results[(l_rate,r_strength)] = (train_accuracy,val_accuracy)
        if val_accuracy> best_val:
            best_val = val_accuracy
            best_softmax = softmax
################################################################################
#                              END OF YOUR CODE                                #
################################################################################
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

In [ ]:
# evaluate on test set
# Evaluate the best softmax on test set
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('softmax on raw pixels final test set accuracy: %f' % (test_accuracy, ))